In [42]:
import pandas as pd
import numpy as np
duda=pd.read_excel('Duda_MSSNG.xlsx',sheet_name="Table S4", skiprows=1)
duda

,SAMPLE,CHROM,START,END,Platform,Validation,REF,ALT,PLATFORM,TYPE,TYPESEQ,EFFECT,GENE_SYMBOL,GENE_ID,OMIM_ID,OMIM_PHENOTYPE
0,1-0006-003,2,98429018,98429019,CG,de novo,C,T,CG,snp,splicing,NaN,TMEM131,23505,615659.0,NaN
1,1-0028-003,10,28903528,28903533,ILMN,de novo,TCACA,TCA,ILMN,del,exonic,frameshift deletion,WAC,51322,615049.0,NaN
2,1-0059-003,19,55686639,55686642,CG,inherited,CGA,C,CG,del,exonic,frameshift deletion,SYT5,6861,600782.0,NaN
3,1-0065-004,7,23191720,23191721,CG,de novo,C,T,CG,snp,exonic,stopgain,KLHL7,55975,611119.0,"Retinitis pigmentosa 42, 612943 (3)"
4,1-0065-005,22,51019860,51019865,CG,inherited,CCAAA,C,CG,del,exonic,frameshift deletion,CHKB,1120,612395.0,"Muscular dystrophy, congenital, megaconial typ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,AU4372309,11,94192626,94192627,ILMN,de novo,G,A,ILMN,snp,exonic,stopgain,MRE11A,4361,600814.0,"Ataxia-telangiectasia-like disorder, 604391 (3)"
222,AU4376301,18,51795940,51795941,ILMN,de novo,G,T,ILMN,snp,exonic,stopgain,POLI,11201,605252.0,NaN
223,AU4392301,11,46897495,46897496,ILMN,de novo,C,T,ILMN,snp,exonic,stopgain,LRP4,4038,604270.0,"?Myasthenic syndrome, congenital, 17, 616304 (..."
224,AU4412302,17,27244344,27244351,ILMN,de novo,AGGGGGG,AGGGGGGG,ILMN,ins,exonic,frameshift insertion,PHF12,57649,NaN,NaN


In [43]:
mssng_sig=["CHD8","ARID1B","SCN2A","DYRK1A","SYNGAP1","ADNP","ANK2","DSCAM","ASH1L","KMT5B","KMT2A",
        "CHD2","GIGYF1","GRIN2B","POGZ","ASXL3","KMT2C","WAC","WDFY3","SHANK3","ANKRD11","CACNA2D3",
        "CUL3","FOXP1","KDM6B","PHF2","SPAST","TBR1","TCF7L2","TNRC6B","CIC","CNOT3","DDX3X","DIP2C",
        "MED13","MYT1L","NAA15","PAX5","PHF3","PTEN","SHANK2","SMARCC2","SRSF11","UBN2","UPF3B","KDM6A",
        "MECP2","PCDH11X","NEXMIF","AFF2","NLGN4X","NLGN3","FAM47A","DYNC1H1","SLC6A1","AGAP2",
        "ADCY3","CLASP1","DNMT3A","MYO5A","TAF6"]

print(len(mssng_sig))

61


In [44]:
novel=['CIC', 'CNOT3', 'DIP2C', 'MED13', 'PAX5', 'PHF3', 'SMARCC2', 'SRSF11', 'UBN2', 'DYNC1H1',
        'AGAP2', 'ADCY3', 'CLASP1', 'MYO5A', 'TAF6', 'PCDH11X', 'NEXMIF','FAM47A']
print(len(novel))  

18


l=[]
ENSG00000079432 --- CIC
ENSG00000088038 --- CNOT3
ENSG00000151240 --- DIP2C
ENSG00000108510 --- MED13
ENSG00000196092 --- PAX5
ENSG00000118482 --- PHF3
ENSG00000139613 --- SMARCC2
ENSG00000116754 --- SRSF11
ENSG00000157741 --- UBN2
ENSG00000197102 --- DYNC1H1
ENSG00000135439 --- AGAP2
ENSG00000138031 --- ADCY3
ENSG00000074054 --- CLASP1
ENSG00000197535 --- MYO5A
ENSG00000106290 --- 
ENSG00000099715 --- TAF6
ENSG00000102290 ---	PCDH11X
ENSG00000050030 --- NEXMIF
ENSG00000185448 --- FAM47A
len(set(l))

## Important
changed KIAA2022 to NEXMIF because it is a synonym and KIAA2022 was not detected with Gprofiler

In [45]:

lof=list(set(duda["GENE_ID"].astype(int).tolist()))

complete_list = duda["GENE_ID"].tolist()

type(lof[0])

int

In [46]:
def gmt_creator(gene_lists, name,query_data_type='ENTREZGENE_ACC'):


    from gprofiler import GProfiler
    gmt_strigs = []
    
    for gene_list, name in zip(gene_lists, name):
        if isinstance(gene_list[0], str):  # Check if the first element of the gene list is a string
            query_data_type = 'SYMBOL'
        else:
            query_data_type = 'ENTREZGENE_ACC'
            
        gp = GProfiler(return_dataframe=True)
        gp_convert_df =gp.convert(organism  = 'hsapiens',
                                    query= gene_list,
                                        target_namespace= 'ENSG',
                                        numeric_namespace=query_data_type,)
        gp_convert_df = gp_convert_df[['incoming', 'converted']]
        gp_convert_df = gp_convert_df.loc[gp_convert_df['converted'] != 'ENSG00000106290'] # X and Y chromosome error, removed the Y chromosome gene, as done before
        ensemble_list = gp_convert_df['converted'].tolist()
        gene_string = '\t'.join(ensemble_list)
        gmt_line= f'{name}\tMutation: {name}\t' + gene_string
        gmt_strigs.append(gmt_line)
        print(gmt_line)

    with open('mssng.gmt', 'w') as f:
        for gmt_line in gmt_strigs:
            f.write(gmt_line + "\n")     


In [47]:
gmt_creator([novel, mssng_sig, lof], ['MSSNG_NOVEL', 'MSSNG_SIG','MSSNG_LOF'])

MSSNG_NOVEL	Mutation: MSSNG_NOVEL	ENSG00000079432	ENSG00000088038	ENSG00000151240	ENSG00000108510	ENSG00000196092	ENSG00000118482	ENSG00000139613	ENSG00000116754	ENSG00000157741	ENSG00000197102	ENSG00000135439	ENSG00000138031	ENSG00000074054	ENSG00000197535	ENSG00000099715	ENSG00000102290	ENSG00000050030	ENSG00000185448
MSSNG_SIG	Mutation: MSSNG_SIG	ENSG00000100888	ENSG00000049618	ENSG00000136531	ENSG00000157540	ENSG00000197283	ENSG00000101126	ENSG00000145362	ENSG00000171587	ENSG00000116539	ENSG00000110066	ENSG00000118058	ENSG00000173575	ENSG00000146830	ENSG00000273079	ENSG00000143442	ENSG00000141431	ENSG00000055609	ENSG00000095787	ENSG00000163625	ENSG00000251322	ENSG00000167522	ENSG00000157445	ENSG00000036257	ENSG00000114861	ENSG00000285708	ENSG00000132510	ENSG00000197724	ENSG00000021574	ENSG00000136535	ENSG00000148737	ENSG00000100354	ENSG00000079432	ENSG00000088038	ENSG00000215301	ENSG00000151240	ENSG00000108510	ENSG00000186487	ENSG00000164134	ENSG00000196092	ENSG00000118482	ENSG0000

# Test Gprofiler
 



        
          
   
    
     
      
       
        
         
          

In [ ]:
results=pd.read_csv('svc_clean.csv')
gene_list=results['Ensembl_ID'].tolist()
len(gene_list)


data = pd.read_csv('svc_clean.csv')# chage data as need be

data["Decile_rank"] = pd.qcut(data["Probability_Class_1"], 10, labels=False)
data["Decile_rank"] = data["Decile_rank"].astype('category')
data["Decile_rank"] = data["Decile_rank"].cat.rename_categories([f'D{i}' for i in range(10,0,-1)])

deciles = {}
for i in range(1, 11):
        deciles[f'd_{i}'] = data[data["Decile_rank"] == f'D{i}'].copy()



d_1=deciles['d_1']['Ensembl_ID'].tolist()
len(d_1)

In [84]:
import requests
import pandas as pd
import numpy as np
from gprofiler import GProfiler

def get_token_form_response(response):
    if response.status_code == 200:
        token = response.json()['organism']
    else:
        try:
            error_message = 'Error: {}'.format(response.json()['message'])
        except:
            error_message = 'Error, status code {}'.format(response.status_code)
        raise AssertionError(error_message)
    #print("Token:", token)
    return token

def main_custom(deciles, gmt_file,mutation_name):


    dataframes= []
    for i,key in enumerate(deciles):
        decile_list = deciles[key]["Ensembl_ID"].to_list()
        print(f'Processing {key,mutation_name}...')

        gp = GProfiler(
            user_agent='gprofiler_custom_gmt', #optional user agent
            return_dataframe=True, #return pandas dataframe or plain python structures    
        )

        with open(gmt_file) as f:
            response = requests.post('https://biit.cs.ut.ee/gprofiler/api/gost/custom/',
                            json={'gmt':f.read(),
                                'name': gmt_file})
        token = get_token_form_response(response)

        decile_results = gp.profile(query=decile_list, organism=token,user_threshold=1,domain_scope="custom",background=d_1)
        decile_results.insert(0, "Decile", i+1)
        decile_results=decile_results[decile_results['native']==mutation_name]
        dataframes.append(decile_results)


    data = pd.concat(dataframes)
    data= data.sort_values(by='Decile', ascending=False).reset_index(drop=True)
    data['Intersection_to_Term_Ratio'] = data['intersection_size'] / data['term_size']
    data=data.sort_values(by='Decile', ascending=True).reset_index(drop=True)

    return data


In [85]:
MSSNG_lof = main_custom(deciles, 'krish_duda.gmt','prb_rec_LGD'),"MSSNG_lof" # native is the first 
MSSNG_lof[0]

Processing ('d_1', 'prb_rec_LGD')...
Processing ('d_2', 'prb_rec_LGD')...
Processing ('d_3', 'prb_rec_LGD')...
Processing ('d_4', 'prb_rec_LGD')...
Processing ('d_5', 'prb_rec_LGD')...
Processing ('d_6', 'prb_rec_LGD')...
Processing ('d_7', 'prb_rec_LGD')...
Processing ('d_8', 'prb_rec_LGD')...
Processing ('d_9', 'prb_rec_LGD')...
Processing ('d_10', 'prb_rec_LGD')...


C:\Users\joaop\AppData\Local\Temp\ipykernel_13580\1555620758.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat(dataframes)


,Decile,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,Intersection_to_Term_Ratio
0,1,krish_duda,prb_rec_LGD,Mutation: prb_rec_LGD,1.0,False,Mutation: prb_rec_LGD,20,291,20,291,0.068729,1.0,query_1,[],1.0
